<a href="https://colab.research.google.com/github/TiFcode/TiFai/blob/main/TiFai_tif3_object_recognition_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('*** START PROGRAM ********************************************************')

outputDetails = False
outputInfo = True
outputEpochs = False
outputConvolutionOutput = False
image_pixels_X = 224
image_pixels_Y = 224
googleDriveFolderPath = '/content/drive/My Drive/Personal/Dev/ChatGPT/2023-01-05 - AI - object recognition/'
# see the cell below for help on setting the value for googleDriveFolderPath

In [ ]:
# To get the value for googleDriveFolderPath, you can use the following code, then comment back this cell.

"""
from google.colab import drive
import glob
drive.mount('/content/drive')
pattern = '/content/drive/My Drive/**/*20230105_151957.jpg'
filenames = glob.glob(pattern, recursive=True)
print(filenames)
"""

In [ ]:
from google.colab.patches import cv2_imshow

def display_image(image, description):
  print(f'{description}:')
  print(f'Image size: {image.shape}')
  cv2_imshow(image)


In [ ]:
import inspect

def retrieve_variable_name_from_local(var):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var]
    
def retrieve_variable_name_from_call(var):
    callers_local_vars = inspect.currentframe().f_back.f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var]

def textForVar(variable):
  return f'{retrieve_variable_name_from_call(variable)} is [{variable}]'

def textForVarWithDesc(variable, description):
  return f'{description} is [{variable}]'

def printInfo(text):
  if outputInfo:
    print(text)

In [ ]:
from google.colab import drive
import glob
!pwd
!ls
# Mount Google Drive
drive.mount('/content/drive')
!pwd
!ls

In [ ]:
import os
import cv2

def loadAndProcessAllFilesFromFolder(folderPath):
  filenames = os.listdir(folderPath)

  imagesArray = []
  for filename in filenames:
    filenameFullPath = folderPath + filename
    if outputDetails:
      print(f'Found file [{filename}] with size [{os.stat(filenameFullPath).st_size}] bytes.')
    
    image = cv2.imread(filenameFullPath)
    if outputDetails:
      display_image(image, filenameFullPath)
    imagesArray.append(image)

    if outputDetails:
      print(f'imagesArray contains [{len(imagesArray)}] elements.')

  # Convert the images to a format that can be used as input to a CNN
  imagesArray = [cv2.cvtColor(image, cv2.COLOR_BGR2RGB) for image in imagesArray]

  return imagesArray

In [ ]:
def load_images(folderName):
  print(f'Loading {folderName}...')
  images = loadAndProcessAllFilesFromFolder(googleDriveFolderPath + folderName + '/')
  print(f'{folderName} contains [{len(images)}] images.')
  return images

training_images_yes = load_images('training_images_yes')
training_images_non = load_images('training_images_non')
validation_images_yes = load_images('validation_images_yes')
validation_images_non = load_images('validation_images_non')
search_images = load_images('search_images')

print('*** FINISH LOADING IMAGES ********************************************************')


In [ ]:
"""
from sklearn.model_selection import train_test_split

# Split the object images automatically into a training set and a validation set
X_train, X_val = train_test_split(object_images, test_size=0.2, random_state=42)
"""

In [ ]:
import numpy as np

def prepare_array(list_of_images):
  print(f'START: Preparing array [{retrieve_variable_name_from_call(list_of_images)}] ...')

  # Convert the list to numpy array
  printInfo(textForVarWithDesc(type(list_of_images), "type(list_of_images)"))
  printInfo(textForVarWithDesc(len(list_of_images), "len(list_of_images)"))
  array_of_images = np.array(list_of_images)
  printInfo(textForVarWithDesc(type(array_of_images), "type(array_of_images)"))
  printInfo(textForVarWithDesc(array_of_images.shape[0], "array_of_images.shape[0]"))

  printInfo(textForVarWithDesc(array_of_images.shape, "array_of_images.shape"))

  print(f'END: Preparing array [{retrieve_variable_name_from_call(list_of_images)}].\n\n\n')

  return array_of_images

# Preparing the image arrays
training_images_yes = prepare_array(training_images_yes)
training_images_non = prepare_array(training_images_non)
validation_images_yes = prepare_array(validation_images_yes)
validation_images_non = prepare_array(validation_images_non)

target_labels_of_training_images_yes = np.ones(training_images_yes.shape[0])
target_labels_of_training_images_non = np.zeros(training_images_non.shape[0])
target_labels_of_validation_images_yes = np.ones(validation_images_yes.shape[0])
target_labels_of_validation_images_non = np.zeros(validation_images_non.shape[0])

training_images = np.concatenate((training_images_yes, training_images_non))
validation_images = np.concatenate((validation_images_yes, validation_images_non))
target_labels_of_training_images = np.concatenate((target_labels_of_training_images_yes, target_labels_of_training_images_non))
target_labels_of_validation_images = np.concatenate((target_labels_of_validation_images_yes, target_labels_of_validation_images_non))

search_images = prepare_array(search_images)

printInfo(textForVarWithDesc(training_images.shape, "training_images.shape"))
printInfo(textForVarWithDesc(validation_images.shape, "validation_images.shape"))
printInfo(textForVarWithDesc(target_labels_of_training_images.shape, "target_labels_of_training_images.shape"))
printInfo(textForVarWithDesc(target_labels_of_validation_images.shape, "target_labels_of_validation_images.shape"))
printInfo(textForVarWithDesc(search_images.shape, "search_images.shape"))


In [ ]:
#@title
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential

# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(image_pixels_X, image_pixels_Y, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
training_batch_size = training_images.shape[0]
if outputEpochs:
  verbose=1
else:
  verbose=0

history = model.fit(training_images, target_labels_of_training_images, batch_size=training_batch_size, epochs=100, validation_data=(validation_images, target_labels_of_validation_images), verbose=verbose)


In [ ]:
model.summary()

To visualize the features extracted by a convolutional neural network (CNN), we can use the matplotlib library to plot the feature maps produced by the filters in the different layers of the CNN.

In this example, the feature maps are extracted from the CNN by creating a new model that maps the input to the feature maps produced by the convolutional layer. The feature maps are then predicted for a given input image and plotted using imshow(). We can adjust the number of rows and columns in the plot by changing the values of 8 and 4 in the subplot() function.

In [ ]:
cv2_imshow(training_images[0]) 

In [ ]:
if outputConvolutionOutput:
  # Import the necessary modules
  import matplotlib.pyplot as plt
  from keras.models import Model

  # Extract the feature maps from the CNN
  feature_maps = model.get_layer('conv2d').output

  # Create a model that maps the input to the feature maps
  feature_map_model = Model(inputs=model.input, outputs=feature_maps)

  # Use the model to predict the feature maps for an input image
  input_image = np.expand_dims(training_images[0], axis=0)
  feature_maps = feature_map_model.predict(input_image)


In [ ]:
if outputConvolutionOutput:
  print(textForVarWithDesc(len(feature_maps[0]), 'len(feature_maps[0])'))

  # Plot the feature maps
  for i, feature_map in enumerate(feature_maps[0]):
    #plt.subplot(56, 4, i + 1)
    print(textForVarWithDesc(i, '\n\nValue of i'))
    #plt.figure(figsize=(10, 5))
    #plt.imshow(feature_map, cmap='gray')

    print(textForVarWithDesc(feature_map.shape, 'feature_map.shape'))
    fig, ax = plt.subplots(1, 1, figsize=(15, 0.5))
    ax.imshow(feature_map, cmap='plasma', aspect='auto')

    plt.show()
    #break

To visualize the training history, we use the matplotlib library to plot the training and validation accuracy and loss over time:

In [ ]:

# Import the necessary modules
import matplotlib.pyplot as plt

# Extract the training and validation accuracy and loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

# Create the plot
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

In [ ]:
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
# Generate predictions for the search images
predictions = model.predict(search_images)

In [ ]:
# Set the threshold
threshold = 0.5

# Loop through the predictions
for i in range(len(predictions)):
  cv2_imshow(search_images[i])
  print(textForVarWithDesc(predictions[i], f'Search image number {i} -> predictions[{i}]'))
  # If the prediction is greater than the threshold, print "Object found"
  if predictions[i] > threshold:
    print(f"=> The confidence is higher than {threshold * 100}% that there is SOMETHING in the search image number {i}\n\t\t\t\t\t\t\tthat is SIMILAR to the training images.\n\n\n\n\n")
  # Otherwise, print "Object not found"
  else:
    print(f"Confidence lower than {threshold * 100}% \n\n\n\n\n")


In [ ]:
print('*** END PROGRAM ********************************************************')
